In [31]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from scipy.stats import chi2
import statsmodels.api as sm
from sklearn.metrics import f1_score, classification_report
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier



In [32]:
df = pd.read_csv("crash.csv")

C:\Users\DELL\AppData\Local\Temp\ipykernel_36260\1007217188.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("crash.csv")


In [33]:
df.head()




,Report Number,Local Case Number,Agency Name,ACRS Report Type,Crash Date/Time,Route Type,Road Name,Cross-Street Type,Cross-Street Name,Off-Road Description,...,Speed Limit,Driverless Vehicle,Parked Vehicle,Vehicle Year,Vehicle Make,Vehicle Model,Equipment Problems,Latitude,Longitude,Location
0,MCP3170003V,240000438,Montgomery County Police,Property Damage Crash,01/03/2024 02:55:00 PM,NaN,NaN,NaN,NaN,IN FRONT OF 18900 BIRDSEYE DR,...,0,No,No,2017,LEXUS,SUV,NO MISUSE,39.165005,-77.249310,"(39.16500483, -77.24931)"
1,MCP3254003K,230072050,Montgomery County Police,Injury Crash,12/16/2023 12:36:00 PM,Maryland (State),GERMANTOWN RD,County,MIDDLEBROOK RD,NaN,...,35,No,No,2010,TOYT,PRIUS,NO MISUSE,39.178776,-77.267190,"(39.17877577, -77.26718974)"
2,EJ7887003Q,230074270,Gaithersburg Police Depar,Injury Crash,12/29/2023 12:00:00 PM,Maryland (State),GREAT SENECA HWY,Municipality,KENTLANDS BLVD,NaN,...,35,No,No,2021,SUBARU,FORRESTER,NO MISUSE,39.123574,-77.231769,"(39.12357374, -77.231769)"
3,MCP2674004J,230064598,Montgomery County Police,Property Damage Crash,11/05/2023 09:07:00 PM,Maryland (State),WOODFIELD RD,County,GLENDALOUGH RD,NaN,...,40,No,No,2019,DODGE,CHARGER,NO MISUSE,39.211742,-77.171461,"(39.21174219, -77.17146065)"
4,MCP25280008,230067019,Montgomery County Police,Property Damage Crash,11/18/2023 12:40:00 AM,Maryland (State),CLARKSBURG RD,Maryland (State),CLARKSBURG RD,NaN,...,20,No,No,2014,NISSAN,ROGUE,NaN,39.228915,-77.289091,"(39.22891483, -77.28909117)"


In [34]:
df.columns

Index(['Report Number', 'Local Case Number', 'Agency Name', 'ACRS Report Type',
       'Crash Date/Time', 'Route Type', 'Road Name', 'Cross-Street Type',
       'Cross-Street Name', 'Off-Road Description', 'Municipality',
       'Related Non-Motorist', 'Collision Type', 'Weather',
       'Surface Condition', 'Light', 'Traffic Control',
       'Driver Substance Abuse', 'Non-Motorist Substance Abuse', 'Person ID',
       'Driver At Fault', 'Injury Severity', 'Circumstance',
       'Driver Distracted By', 'Drivers License State', 'Vehicle ID',
       'Vehicle Damage Extent', 'Vehicle First Impact Location',
       'Vehicle Second Impact Location', 'Vehicle Body Type',
       'Vehicle Movement', 'Vehicle Continuing Dir', 'Vehicle Going Dir',
       'Speed Limit', 'Driverless Vehicle', 'Parked Vehicle', 'Vehicle Year',
       'Vehicle Make', 'Vehicle Model', 'Equipment Problems', 'Latitude',
       'Longitude', 'Location'],
      dtype='object')

In [35]:
df_cleaned = df.drop(columns=['Report Number', 'Local Case Number', 'Agency Name','Road Name','Cross-Street Name','Off-Road Description', 'Municipality',
       'Related Non-Motorist','Non-Motorist Substance Abuse', 'Person ID','Driver At Fault','Injury Severity', 'Circumstance','Drivers License State','Vehicle ID','Vehicle Damage Extent','Driverless Vehicle', 'Parked Vehicle','Equipment Problems', 'Latitude',
       'Longitude', 'Location'])




In [36]:
df_cleaned.columns

Index(['ACRS Report Type', 'Crash Date/Time', 'Route Type',
       'Cross-Street Type', 'Collision Type', 'Weather', 'Surface Condition',
       'Light', 'Traffic Control', 'Driver Substance Abuse',
       'Driver Distracted By', 'Vehicle First Impact Location',
       'Vehicle Second Impact Location', 'Vehicle Body Type',
       'Vehicle Movement', 'Vehicle Continuing Dir', 'Vehicle Going Dir',
       'Speed Limit', 'Vehicle Year', 'Vehicle Make', 'Vehicle Model'],
      dtype='object')

In [37]:
df_cleaned.dropna()

,ACRS Report Type,Crash Date/Time,Route Type,Cross-Street Type,Collision Type,Weather,Surface Condition,Light,Traffic Control,Driver Substance Abuse,...,Vehicle First Impact Location,Vehicle Second Impact Location,Vehicle Body Type,Vehicle Movement,Vehicle Continuing Dir,Vehicle Going Dir,Speed Limit,Vehicle Year,Vehicle Make,Vehicle Model
1,Injury Crash,12/16/2023 12:36:00 PM,Maryland (State),County,STRAIGHT MOVEMENT ANGLE,CLEAR,DRY,DAYLIGHT,TRAFFIC SIGNAL,NONE DETECTED,...,ELEVEN OCLOCK,ELEVEN OCLOCK,PASSENGER CAR,MOVING CONSTANT SPEED,North,West,35,2010,TOYT,PRIUS
2,Injury Crash,12/29/2023 12:00:00 PM,Maryland (State),Municipality,SAME DIR REAR END,CLEAR,DRY,DAYLIGHT,TRAFFIC SIGNAL,NONE DETECTED,...,SIX OCLOCK,SIX OCLOCK,(SPORT) UTILITY VEHICLE,MOVING CONSTANT SPEED,South,South,35,2021,SUBARU,FORRESTER
3,Property Damage Crash,11/05/2023 09:07:00 PM,Maryland (State),County,SINGLE VEHICLE,CLEAR,DRY,DARK LIGHTS ON,NO CONTROLS,NONE DETECTED,...,ONE OCLOCK,ONE OCLOCK,POLICE VEHICLE/NON EMERGENCY,MOVING CONSTANT SPEED,South,South,40,2019,DODGE,CHARGER
4,Property Damage Crash,11/18/2023 12:40:00 AM,Maryland (State),Maryland (State),SINGLE VEHICLE,RAINING,WET,DARK LIGHTS ON,YIELD SIGN,NONE DETECTED,...,ELEVEN OCLOCK,ELEVEN OCLOCK,PASSENGER CAR,MOVING CONSTANT SPEED,North,North,20,2014,NISSAN,ROGUE
5,Injury Crash,12/29/2023 12:00:00 PM,Maryland (State),Municipality,SAME DIR REAR END,CLEAR,DRY,DAYLIGHT,TRAFFIC SIGNAL,NONE DETECTED,...,TWELVE OCLOCK,TWELVE OCLOCK,PASSENGER CAR,MOVING CONSTANT SPEED,South,South,35,1991,MERCEDES-BENZ,L350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172094,Injury Crash,02/25/2019 07:55:00 AM,County,Maryland (State),SAME DIR REAR END,CLEAR,DRY,DAYLIGHT,TRAFFIC SIGNAL,NONE DETECTED,...,SIX OCLOCK,SIX OCLOCK,PASSENGER CAR,STOPPED IN TRAFFIC LANE,South,South,35,2015,LINC,MKZ
172098,Property Damage Crash,10/29/2022 06:01:00 AM,Maryland (State),Maryland (State),STRAIGHT MOVEMENT ANGLE,CLEAR,DRY,DARK LIGHTS ON,TRAFFIC SIGNAL,NONE DETECTED,...,TWELVE OCLOCK,TWELVE OCLOCK,PASSENGER CAR,MOVING CONSTANT SPEED,West,West,25,2017,CHEVY,EQUINOX
172099,Property Damage Crash,03/25/2018 02:10:00 PM,Municipality,Municipality,STRAIGHT MOVEMENT ANGLE,CLEAR,DRY,DAYLIGHT,STOP SIGN,NONE DETECTED,...,EIGHT OCLOCK,EIGHT OCLOCK,(SPORT) UTILITY VEHICLE,MOVING CONSTANT SPEED,North,North,25,2008,CHEVROLET,EQUINOX
172102,Property Damage Crash,11/23/2020 07:37:00 AM,Maryland (State),Unknown,SAME DIR REAR END,CLOUDY,DRY,DAYLIGHT,NO CONTROLS,NONE DETECTED,...,SIX OCLOCK,SIX OCLOCK,VAN,SLOWING OR STOPPING,North,North,40,2008,CHEVROLET,EXPRESS


In [38]:
columns = df_cleaned.columns
columns

Index(['ACRS Report Type', 'Crash Date/Time', 'Route Type',
       'Cross-Street Type', 'Collision Type', 'Weather', 'Surface Condition',
       'Light', 'Traffic Control', 'Driver Substance Abuse',
       'Driver Distracted By', 'Vehicle First Impact Location',
       'Vehicle Second Impact Location', 'Vehicle Body Type',
       'Vehicle Movement', 'Vehicle Continuing Dir', 'Vehicle Going Dir',
       'Speed Limit', 'Vehicle Year', 'Vehicle Make', 'Vehicle Model'],
      dtype='object')

In [39]:
for column in columns:
  if df_cleaned[column].dtype == 'String':
    df_cleaned = df_cleaned[~ df_cleaned[column].str.contains('unknown', case=False
)]

In [40]:
df_cleaned

,ACRS Report Type,Crash Date/Time,Route Type,Cross-Street Type,Collision Type,Weather,Surface Condition,Light,Traffic Control,Driver Substance Abuse,...,Vehicle First Impact Location,Vehicle Second Impact Location,Vehicle Body Type,Vehicle Movement,Vehicle Continuing Dir,Vehicle Going Dir,Speed Limit,Vehicle Year,Vehicle Make,Vehicle Model
0,Property Damage Crash,01/03/2024 02:55:00 PM,NaN,NaN,OPPOSITE DIRECTION SIDESWIPE,CLOUDY,NaN,DAYLIGHT,NO CONTROLS,NONE DETECTED,...,SEVEN OCLOCK,SEVEN OCLOCK,PASSENGER CAR,MOVING CONSTANT SPEED,South,South,0,2017,LEXUS,SUV
1,Injury Crash,12/16/2023 12:36:00 PM,Maryland (State),County,STRAIGHT MOVEMENT ANGLE,CLEAR,DRY,DAYLIGHT,TRAFFIC SIGNAL,NONE DETECTED,...,ELEVEN OCLOCK,ELEVEN OCLOCK,PASSENGER CAR,MOVING CONSTANT SPEED,North,West,35,2010,TOYT,PRIUS
2,Injury Crash,12/29/2023 12:00:00 PM,Maryland (State),Municipality,SAME DIR REAR END,CLEAR,DRY,DAYLIGHT,TRAFFIC SIGNAL,NONE DETECTED,...,SIX OCLOCK,SIX OCLOCK,(SPORT) UTILITY VEHICLE,MOVING CONSTANT SPEED,South,South,35,2021,SUBARU,FORRESTER
3,Property Damage Crash,11/05/2023 09:07:00 PM,Maryland (State),County,SINGLE VEHICLE,CLEAR,DRY,DARK LIGHTS ON,NO CONTROLS,NONE DETECTED,...,ONE OCLOCK,ONE OCLOCK,POLICE VEHICLE/NON EMERGENCY,MOVING CONSTANT SPEED,South,South,40,2019,DODGE,CHARGER
4,Property Damage Crash,11/18/2023 12:40:00 AM,Maryland (State),Maryland (State),SINGLE VEHICLE,RAINING,WET,DARK LIGHTS ON,YIELD SIGN,NONE DETECTED,...,ELEVEN OCLOCK,ELEVEN OCLOCK,PASSENGER CAR,MOVING CONSTANT SPEED,North,North,20,2014,NISSAN,ROGUE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172100,Property Damage Crash,03/01/2016 10:01:00 AM,Municipality,Municipality,OTHER,CLEAR,DRY,DAYLIGHT,NaN,NaN,...,ONE OCLOCK,ONE OCLOCK,POLICE VEHICLE/EMERGENCY,SLOWING OR STOPPING,East,East,25,2016,FORD,EXPLORER
172101,Property Damage Crash,07/19/2017 02:22:00 PM,County,Maryland (State),SAME DIR REAR END,CLEAR,DRY,DAYLIGHT,NO CONTROLS,NaN,...,TWELVE OCLOCK,TWELVE OCLOCK,PASSENGER CAR,SLOWING OR STOPPING,East,East,35,2008,NISSAN,ALTIMA
172102,Property Damage Crash,11/23/2020 07:37:00 AM,Maryland (State),Unknown,SAME DIR REAR END,CLOUDY,DRY,DAYLIGHT,NO CONTROLS,NONE DETECTED,...,SIX OCLOCK,SIX OCLOCK,VAN,SLOWING OR STOPPING,North,North,40,2008,CHEVROLET,EXPRESS
172103,Property Damage Crash,11/23/2019 11:23:00 PM,County,Ramp,ANGLE MEETS LEFT TURN,RAINING,WET,DARK LIGHTS ON,TRAFFIC SIGNAL,NONE DETECTED,...,ONE OCLOCK,ONE OCLOCK,VAN,MAKING LEFT TURN,East,South,35,2018,FORD,TRANSIT VAN


In [41]:
df = df_cleaned.dropna()

In [42]:
df.head(10)

,ACRS Report Type,Crash Date/Time,Route Type,Cross-Street Type,Collision Type,Weather,Surface Condition,Light,Traffic Control,Driver Substance Abuse,...,Vehicle First Impact Location,Vehicle Second Impact Location,Vehicle Body Type,Vehicle Movement,Vehicle Continuing Dir,Vehicle Going Dir,Speed Limit,Vehicle Year,Vehicle Make,Vehicle Model
1,Injury Crash,12/16/2023 12:36:00 PM,Maryland (State),County,STRAIGHT MOVEMENT ANGLE,CLEAR,DRY,DAYLIGHT,TRAFFIC SIGNAL,NONE DETECTED,...,ELEVEN OCLOCK,ELEVEN OCLOCK,PASSENGER CAR,MOVING CONSTANT SPEED,North,West,35,2010,TOYT,PRIUS
2,Injury Crash,12/29/2023 12:00:00 PM,Maryland (State),Municipality,SAME DIR REAR END,CLEAR,DRY,DAYLIGHT,TRAFFIC SIGNAL,NONE DETECTED,...,SIX OCLOCK,SIX OCLOCK,(SPORT) UTILITY VEHICLE,MOVING CONSTANT SPEED,South,South,35,2021,SUBARU,FORRESTER
3,Property Damage Crash,11/05/2023 09:07:00 PM,Maryland (State),County,SINGLE VEHICLE,CLEAR,DRY,DARK LIGHTS ON,NO CONTROLS,NONE DETECTED,...,ONE OCLOCK,ONE OCLOCK,POLICE VEHICLE/NON EMERGENCY,MOVING CONSTANT SPEED,South,South,40,2019,DODGE,CHARGER
4,Property Damage Crash,11/18/2023 12:40:00 AM,Maryland (State),Maryland (State),SINGLE VEHICLE,RAINING,WET,DARK LIGHTS ON,YIELD SIGN,NONE DETECTED,...,ELEVEN OCLOCK,ELEVEN OCLOCK,PASSENGER CAR,MOVING CONSTANT SPEED,North,North,20,2014,NISSAN,ROGUE
5,Injury Crash,12/29/2023 12:00:00 PM,Maryland (State),Municipality,SAME DIR REAR END,CLEAR,DRY,DAYLIGHT,TRAFFIC SIGNAL,NONE DETECTED,...,TWELVE OCLOCK,TWELVE OCLOCK,PASSENGER CAR,MOVING CONSTANT SPEED,South,South,35,1991,MERCEDES-BENZ,L350
6,Injury Crash,12/17/2023 05:43:00 PM,Maryland (State),US (State),STRAIGHT MOVEMENT ANGLE,RAINING,WET,DARK LIGHTS ON,TRAFFIC SIGNAL,NONE DETECTED,...,TWELVE OCLOCK,TWELVE OCLOCK,PASSENGER CAR,MAKING LEFT TURN,South,West,35,2015,TOYOTA,COROLLA
10,Property Damage Crash,12/26/2023 11:05:00 PM,County,County,OTHER,RAINING,WET,DARK LIGHTS ON,NO CONTROLS,NONE DETECTED,...,EIGHT OCLOCK,EIGHT OCLOCK,POLICE VEHICLE/EMERGENCY,MOVING CONSTANT SPEED,East,East,25,2018,FORD,INTERCEPTOR
14,Property Damage Crash,12/24/2023 08:00:00 PM,Maryland (State),County,SAME DIR REAR END,CLEAR,DRY,DARK NO LIGHTS,NO CONTROLS,NONE DETECTED,...,SIX OCLOCK,SIX OCLOCK,PASSENGER CAR,SLOWING OR STOPPING,South,South,45,2023,TOYOTA,COROLLA
15,Property Damage Crash,12/12/2023 08:20:00 PM,County,County,SAME DIRECTION SIDESWIPE,CLEAR,DRY,DARK NO LIGHTS,NO CONTROLS,NONE DETECTED,...,TWO OCLOCK,TWO OCLOCK,PASSENGER CAR,MOVING CONSTANT SPEED,North,North,25,2012,KIA,SEDONA
16,Property Damage Crash,12/24/2023 08:00:00 PM,Maryland (State),County,SAME DIR REAR END,CLEAR,DRY,DARK NO LIGHTS,NO CONTROLS,NONE DETECTED,...,TWELVE OCLOCK,TWELVE OCLOCK,PASSENGER CAR,MOVING CONSTANT SPEED,South,South,45,2017,JEEP,CHEROKEE


In [43]:
df.columns

Index(['ACRS Report Type', 'Crash Date/Time', 'Route Type',
       'Cross-Street Type', 'Collision Type', 'Weather', 'Surface Condition',
       'Light', 'Traffic Control', 'Driver Substance Abuse',
       'Driver Distracted By', 'Vehicle First Impact Location',
       'Vehicle Second Impact Location', 'Vehicle Body Type',
       'Vehicle Movement', 'Vehicle Continuing Dir', 'Vehicle Going Dir',
       'Speed Limit', 'Vehicle Year', 'Vehicle Make', 'Vehicle Model'],
      dtype='object')

In [44]:
print(df['ACRS Report Type'].unique())


['Injury Crash' 'Property Damage Crash' 'Fatal Crash']


In [45]:
df['ACRS Report Type'] = df['ACRS Report Type'].replace({'Injury Crash': 1, 'Fatal Crash': 1, 'Property Damage Crash': 0})


C:\Users\DELL\AppData\Local\Temp\ipykernel_36260\4170820224.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ACRS Report Type'] = df['ACRS Report Type'].replace({'Injury Crash': 1, 'Fatal Crash': 1, 'Property Damage Crash': 0})


In [46]:
df.head()

,ACRS Report Type,Crash Date/Time,Route Type,Cross-Street Type,Collision Type,Weather,Surface Condition,Light,Traffic Control,Driver Substance Abuse,...,Vehicle First Impact Location,Vehicle Second Impact Location,Vehicle Body Type,Vehicle Movement,Vehicle Continuing Dir,Vehicle Going Dir,Speed Limit,Vehicle Year,Vehicle Make,Vehicle Model
1,1,12/16/2023 12:36:00 PM,Maryland (State),County,STRAIGHT MOVEMENT ANGLE,CLEAR,DRY,DAYLIGHT,TRAFFIC SIGNAL,NONE DETECTED,...,ELEVEN OCLOCK,ELEVEN OCLOCK,PASSENGER CAR,MOVING CONSTANT SPEED,North,West,35,2010,TOYT,PRIUS
2,1,12/29/2023 12:00:00 PM,Maryland (State),Municipality,SAME DIR REAR END,CLEAR,DRY,DAYLIGHT,TRAFFIC SIGNAL,NONE DETECTED,...,SIX OCLOCK,SIX OCLOCK,(SPORT) UTILITY VEHICLE,MOVING CONSTANT SPEED,South,South,35,2021,SUBARU,FORRESTER
3,0,11/05/2023 09:07:00 PM,Maryland (State),County,SINGLE VEHICLE,CLEAR,DRY,DARK LIGHTS ON,NO CONTROLS,NONE DETECTED,...,ONE OCLOCK,ONE OCLOCK,POLICE VEHICLE/NON EMERGENCY,MOVING CONSTANT SPEED,South,South,40,2019,DODGE,CHARGER
4,0,11/18/2023 12:40:00 AM,Maryland (State),Maryland (State),SINGLE VEHICLE,RAINING,WET,DARK LIGHTS ON,YIELD SIGN,NONE DETECTED,...,ELEVEN OCLOCK,ELEVEN OCLOCK,PASSENGER CAR,MOVING CONSTANT SPEED,North,North,20,2014,NISSAN,ROGUE
5,1,12/29/2023 12:00:00 PM,Maryland (State),Municipality,SAME DIR REAR END,CLEAR,DRY,DAYLIGHT,TRAFFIC SIGNAL,NONE DETECTED,...,TWELVE OCLOCK,TWELVE OCLOCK,PASSENGER CAR,MOVING CONSTANT SPEED,South,South,35,1991,MERCEDES-BENZ,L350


In [47]:
df = df.rename(columns={'ACRS Report Type': 'Injury'})

In [48]:
injury = df['Injury']

In [49]:
df

,Injury,Crash Date/Time,Route Type,Cross-Street Type,Collision Type,Weather,Surface Condition,Light,Traffic Control,Driver Substance Abuse,...,Vehicle First Impact Location,Vehicle Second Impact Location,Vehicle Body Type,Vehicle Movement,Vehicle Continuing Dir,Vehicle Going Dir,Speed Limit,Vehicle Year,Vehicle Make,Vehicle Model
1,1,12/16/2023 12:36:00 PM,Maryland (State),County,STRAIGHT MOVEMENT ANGLE,CLEAR,DRY,DAYLIGHT,TRAFFIC SIGNAL,NONE DETECTED,...,ELEVEN OCLOCK,ELEVEN OCLOCK,PASSENGER CAR,MOVING CONSTANT SPEED,North,West,35,2010,TOYT,PRIUS
2,1,12/29/2023 12:00:00 PM,Maryland (State),Municipality,SAME DIR REAR END,CLEAR,DRY,DAYLIGHT,TRAFFIC SIGNAL,NONE DETECTED,...,SIX OCLOCK,SIX OCLOCK,(SPORT) UTILITY VEHICLE,MOVING CONSTANT SPEED,South,South,35,2021,SUBARU,FORRESTER
3,0,11/05/2023 09:07:00 PM,Maryland (State),County,SINGLE VEHICLE,CLEAR,DRY,DARK LIGHTS ON,NO CONTROLS,NONE DETECTED,...,ONE OCLOCK,ONE OCLOCK,POLICE VEHICLE/NON EMERGENCY,MOVING CONSTANT SPEED,South,South,40,2019,DODGE,CHARGER
4,0,11/18/2023 12:40:00 AM,Maryland (State),Maryland (State),SINGLE VEHICLE,RAINING,WET,DARK LIGHTS ON,YIELD SIGN,NONE DETECTED,...,ELEVEN OCLOCK,ELEVEN OCLOCK,PASSENGER CAR,MOVING CONSTANT SPEED,North,North,20,2014,NISSAN,ROGUE
5,1,12/29/2023 12:00:00 PM,Maryland (State),Municipality,SAME DIR REAR END,CLEAR,DRY,DAYLIGHT,TRAFFIC SIGNAL,NONE DETECTED,...,TWELVE OCLOCK,TWELVE OCLOCK,PASSENGER CAR,MOVING CONSTANT SPEED,South,South,35,1991,MERCEDES-BENZ,L350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172094,1,02/25/2019 07:55:00 AM,County,Maryland (State),SAME DIR REAR END,CLEAR,DRY,DAYLIGHT,TRAFFIC SIGNAL,NONE DETECTED,...,SIX OCLOCK,SIX OCLOCK,PASSENGER CAR,STOPPED IN TRAFFIC LANE,South,South,35,2015,LINC,MKZ
172098,0,10/29/2022 06:01:00 AM,Maryland (State),Maryland (State),STRAIGHT MOVEMENT ANGLE,CLEAR,DRY,DARK LIGHTS ON,TRAFFIC SIGNAL,NONE DETECTED,...,TWELVE OCLOCK,TWELVE OCLOCK,PASSENGER CAR,MOVING CONSTANT SPEED,West,West,25,2017,CHEVY,EQUINOX
172099,0,03/25/2018 02:10:00 PM,Municipality,Municipality,STRAIGHT MOVEMENT ANGLE,CLEAR,DRY,DAYLIGHT,STOP SIGN,NONE DETECTED,...,EIGHT OCLOCK,EIGHT OCLOCK,(SPORT) UTILITY VEHICLE,MOVING CONSTANT SPEED,North,North,25,2008,CHEVROLET,EQUINOX
172102,0,11/23/2020 07:37:00 AM,Maryland (State),Unknown,SAME DIR REAR END,CLOUDY,DRY,DAYLIGHT,NO CONTROLS,NONE DETECTED,...,SIX OCLOCK,SIX OCLOCK,VAN,SLOWING OR STOPPING,North,North,40,2008,CHEVROLET,EXPRESS


In [50]:
df[['Date', 'Time_Period']] = df['Crash Date/Time'].str.split(' ', n = 1, expand=True)

In [51]:
df

,Injury,Crash Date/Time,Route Type,Cross-Street Type,Collision Type,Weather,Surface Condition,Light,Traffic Control,Driver Substance Abuse,...,Vehicle Body Type,Vehicle Movement,Vehicle Continuing Dir,Vehicle Going Dir,Speed Limit,Vehicle Year,Vehicle Make,Vehicle Model,Date,Time_Period
1,1,12/16/2023 12:36:00 PM,Maryland (State),County,STRAIGHT MOVEMENT ANGLE,CLEAR,DRY,DAYLIGHT,TRAFFIC SIGNAL,NONE DETECTED,...,PASSENGER CAR,MOVING CONSTANT SPEED,North,West,35,2010,TOYT,PRIUS,12/16/2023,12:36:00 PM
2,1,12/29/2023 12:00:00 PM,Maryland (State),Municipality,SAME DIR REAR END,CLEAR,DRY,DAYLIGHT,TRAFFIC SIGNAL,NONE DETECTED,...,(SPORT) UTILITY VEHICLE,MOVING CONSTANT SPEED,South,South,35,2021,SUBARU,FORRESTER,12/29/2023,12:00:00 PM
3,0,11/05/2023 09:07:00 PM,Maryland (State),County,SINGLE VEHICLE,CLEAR,DRY,DARK LIGHTS ON,NO CONTROLS,NONE DETECTED,...,POLICE VEHICLE/NON EMERGENCY,MOVING CONSTANT SPEED,South,South,40,2019,DODGE,CHARGER,11/05/2023,09:07:00 PM
4,0,11/18/2023 12:40:00 AM,Maryland (State),Maryland (State),SINGLE VEHICLE,RAINING,WET,DARK LIGHTS ON,YIELD SIGN,NONE DETECTED,...,PASSENGER CAR,MOVING CONSTANT SPEED,North,North,20,2014,NISSAN,ROGUE,11/18/2023,12:40:00 AM
5,1,12/29/2023 12:00:00 PM,Maryland (State),Municipality,SAME DIR REAR END,CLEAR,DRY,DAYLIGHT,TRAFFIC SIGNAL,NONE DETECTED,...,PASSENGER CAR,MOVING CONSTANT SPEED,South,South,35,1991,MERCEDES-BENZ,L350,12/29/2023,12:00:00 PM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172094,1,02/25/2019 07:55:00 AM,County,Maryland (State),SAME DIR REAR END,CLEAR,DRY,DAYLIGHT,TRAFFIC SIGNAL,NONE DETECTED,...,PASSENGER CAR,STOPPED IN TRAFFIC LANE,South,South,35,2015,LINC,MKZ,02/25/2019,07:55:00 AM
172098,0,10/29/2022 06:01:00 AM,Maryland (State),Maryland (State),STRAIGHT MOVEMENT ANGLE,CLEAR,DRY,DARK LIGHTS ON,TRAFFIC SIGNAL,NONE DETECTED,...,PASSENGER CAR,MOVING CONSTANT SPEED,West,West,25,2017,CHEVY,EQUINOX,10/29/2022,06:01:00 AM
172099,0,03/25/2018 02:10:00 PM,Municipality,Municipality,STRAIGHT MOVEMENT ANGLE,CLEAR,DRY,DAYLIGHT,STOP SIGN,NONE DETECTED,...,(SPORT) UTILITY VEHICLE,MOVING CONSTANT SPEED,North,North,25,2008,CHEVROLET,EQUINOX,03/25/2018,02:10:00 PM
172102,0,11/23/2020 07:37:00 AM,Maryland (State),Unknown,SAME DIR REAR END,CLOUDY,DRY,DAYLIGHT,NO CONTROLS,NONE DETECTED,...,VAN,SLOWING OR STOPPING,North,North,40,2008,CHEVROLET,EXPRESS,11/23/2020,07:37:00 AM


In [52]:
def categorize_time(time_str):
    hour = int(time_str.split(':')[0])
    meridiem = time_str.split()[-1]  # Extract the AM/PM part
    if meridiem == 'AM':
        if (0<= hour < 4) or hour == 12 :
            return 'Midnight'
        else:
            return 'Morning'
    
    elif meridiem == 'PM':
        if hour == 12:
            return 'Afternoon'
        elif 6 <= hour < 8:
            return 'Evening'
        else:
            return 'Night'



In [53]:
df['Time_Period'] = df['Time_Period'].apply(categorize_time)


In [54]:
df['Date'] = pd.to_datetime(df['Date'])

In [55]:
df['Day_of_the_week'] = df['Date'].dt.day_name()

In [56]:
df.drop(columns=['Crash Date/Time',  'Date'], inplace=True)

In [57]:
y = df['Injury']
X = df.drop(columns = ['Injury'])

In [58]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.3, random_state= 100)

In [59]:
categorial_col = X.select_dtypes(include= ['object']).columns
num_col = X.select_dtypes(include= ['int','float']).columns


In [60]:
preprocessor = ColumnTransformer(
  transformers= [ 
                 ('num', StandardScaler(), num_col),
                  ('cat', OneHotEncoder(handle_unknown= 'ignore'), categorial_col)])


In [61]:
logistic_regression = LogisticRegression(penalty='l1', solver='liblinear')


pipeline = Pipeline(steps= [('preprocessor', preprocessor),
                           
                           
                            ('classifier',logistic_regression )])


In [62]:
pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  Index(['Speed Limit', 'Vehicle Year'], dtype='object')),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['Route Type', 'Cross-Street Type', 'Collision Type', 'Weather',
       'Surface Condition', 'Light', 'Traffic Control',
       'Driver Substance Abuse', 'Driver Distracted By',
       'Vehicle First Impact Location', 'Vehicle Second Impact Location',
       'Vehicle Body Type', 'Vehicle Movement', 'Vehicle Continuing Dir',
       'Vehicle Going Dir', 'Vehicle Make', 'Vehicle Model', 'Time_Period',
       'Day_of_the_week'],
      dtype='object'))])),
                ('classifier',
                 LogisticRegression(penalty='l1', solver='liblinear'))])

In [63]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  Index(['Speed Limit', 'Vehicle Year'], dtype='object')),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['Route Type', 'Cross-Street Type', 'Collision Type', 'Weather',
       'Surface Condition', 'Light', 'Traffic Control',
       'Driver Substance Abuse', 'Driver Distracted By',
       'Vehicle First Impact Location', 'Vehicle Second Impact Location',
       'Vehicle Body Type', 'Vehicle Movement', 'Vehicle Continuing Dir',
       'Vehicle Going Dir', 'Vehicle Make', 'Vehicle Model', 'Time_Period',
       'Day_of_the_week'],
      dtype='object'))])),
                ('classifier',
                 LogisticRegression(penalty='l1', solver='liblinear'))])

In [64]:
pipeline.score(X_test,y_test)

0.6209444371469854

In [65]:
logistic_model = pipeline.named_steps['classifier']
coef = logistic_model.coef_
cat_feat = pipeline.named_steps['preprocessor'].transformers_[1][1].get_feature_names_out(input_features = categorial_col)
all_feat = np.concatenate((num_col, cat_feat))


In [66]:
coef_df = pd.DataFrame({'Feature': all_feat, 'Coefficient': coef[0]})

In [67]:
coef_df


,Feature,Coefficient
0,Speed Limit,0.117902
1,Vehicle Year,-0.009204
2,Route Type_County,0.018538
3,Route Type_Government,-0.018388
4,Route Type_Interstate (State),0.000000
...,...,...
5244,Day_of_the_week_Saturday,-0.023854
5245,Day_of_the_week_Sunday,-0.023345
5246,Day_of_the_week_Thursday,0.000000
5247,Day_of_the_week_Tuesday,0.048723


In [68]:
coef_df = coef_df[coef_df['Coefficient'] != 0]
coef_df = coef_df.sort_values(by  = 'Coefficient', ascending= False)
coef_df

,Feature,Coefficient
157,Vehicle Body Type_MOPED,2.076623
2512,Vehicle Model_DIESEL,1.994337
2605,Vehicle Model_ECO SPORT,1.940978
158,Vehicle Body Type_MOTORCYCLE,1.622288
3930,Vehicle Model_POLICE INTERCEPTER,1.487498
...,...,...
980,Vehicle Make_PIERCE,-1.509342
4197,Vehicle Model_RS4,-1.590191
1201,Vehicle Make_TRUM,-1.592302
982,Vehicle Make_PIRC,-1.772795


In [69]:
cv_scores = cross_val_score(pipeline,X_test, y_test, cv=5) 

In [70]:
np.mean(cv_scores)

0.6168727932864958

In [71]:
y_pred = pipeline.predict(X_test)

In [72]:
f1 = f1_score(y_test,y_pred)
f1

0.4310710237074276

In [73]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.65      0.79      0.72     18290
           1       0.54      0.36      0.43     12162

    accuracy                           0.62     30452
   macro avg       0.59      0.58      0.57     30452
weighted avg       0.61      0.62      0.60     30452

